# VQ-GAN vs Efficient-VQGAN: 완전 자동 파이프라인

이 노트북 하나로 다음 모든 작업을 수행합니다:
1. ✅ 데이터 준비
2. 🔥 모델 학습 (VQ-GAN + Efficient-VQGAN)
3. 📊 성능 벤치마크
4. 📈 비교 시각화
5. 📄 리포트 생성

**실행 방법:** 위에서부터 순서대로 셀을 실행하세요!

## 1. 환경 설정 및 Import

In [ ]:
# 필요한 라이브러리 설치 확인
import sys
!{sys.executable} -m pip install -q torch torchvision tqdm matplotlib seaborn pandas pillow einops

print("✓ 라이브러리 설치 완료!")

In [ ]:
# Import 모든 필요한 모듈
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from torchvision.utils import make_grid, save_image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import time
import json
from pathlib import Path
import seaborn as sns
from IPython.display import display, HTML, Image as IPImage
import warnings
warnings.filterwarnings('ignore')

# 스타일 설정
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    plt.style.use('seaborn-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ Import 완료!")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# 모델 import
from vqgan.model import VQGAN, Discriminator as VQGANDiscriminator
from efficient_vqgan.model import EfficientVQGAN, MultiScaleDiscriminator
from utils.logger import TrainingLogger
from utils.metrics import calculate_psnr, calculate_ssim

print("✓ 모델 모듈 로드 완료!")

## 2. 설정 (여기서 학습 파라미터 조정)

In [ ]:
# ==================== 학습 설정 ====================
CONFIG = {
    # 기본 설정
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'num_epochs': 5,  # 빠른 데모를 위해 5 epoch (실제: 50-100)
    'image_size': 256,
    
    # 데이터
    'dataset': 'cifar10',
    'batch_size_vqgan': 4,
    'batch_size_efficient': 8,
    'num_workers': 0,  # macOS/Jupyter에서 안정성을 위해 0으로 설정
    
    # 학습
    'lr': 4.5e-6,
    'disc_start': 500,  # Discriminator 시작 step
    'disc_weight': 0.8,
    'perceptual_weight': 1.0,
    
    # 로깅
    'log_interval': 100,  # 몇 step마다 로그
    'save_interval': 1,   # 몇 epoch마다 저장
}

print("="*60)
print("학습 설정:")
print("="*60)
for key, value in CONFIG.items():
    print(f"  {key:25s}: {value}")
print("="*60)

## 3. 데이터 준비

In [ ]:
# 데이터 로딩
print("데이터 로딩 중...")

transform = transforms.Compose([
    transforms.Resize(CONFIG['image_size']),
    transforms.CenterCrop(CONFIG['image_size']),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

train_dataset = datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

val_dataset = datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

print(f"✓ 학습 데이터: {len(train_dataset)} 이미지")
print(f"✓ 검증 데이터: {len(val_dataset)} 이미지")

# 샘플 이미지 확인
fig, axes = plt.subplots(2, 8, figsize=(16, 4))
for i in range(16):
    img, _ = train_dataset[i]
    img = (img + 1) / 2  # denormalize
    axes[i//8, i%8].imshow(img.permute(1, 2, 0))
    axes[i//8, i%8].axis('off')
plt.suptitle('샘플 이미지', fontsize=14)
plt.tight_layout()
plt.show()

## 4. 헬퍼 함수 정의

In [ ]:
# Perceptual Loss
class SimpleLPIPS(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x, y):
        # Simple L2 perceptual loss (LPIPS 대체)
        return torch.mean((x - y) ** 2)

def hinge_d_loss(logits_real, logits_fake):
    loss_real = torch.mean(torch.relu(1. - logits_real))
    loss_fake = torch.mean(torch.relu(1. + logits_fake))
    return 0.5 * (loss_real + loss_fake)

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

def show_reconstruction(original, reconstructed, n=8):
    """원본과 복원 이미지 비교"""
    fig, axes = plt.subplots(2, n, figsize=(n*2, 4))
    
    for i in range(n):
        # Original
        img_orig = (original[i].cpu() + 1) / 2
        axes[0, i].imshow(img_orig.permute(1, 2, 0).clamp(0, 1))
        axes[0, i].axis('off')
        if i == 0:
            axes[0, i].set_title('Original', fontsize=10)
        
        # Reconstructed
        img_recon = (reconstructed[i].cpu() + 1) / 2
        axes[1, i].imshow(img_recon.permute(1, 2, 0).clamp(0, 1))
        axes[1, i].axis('off')
        if i == 0:
            axes[1, i].set_title('Reconstructed', fontsize=10)
    
    plt.tight_layout()
    plt.show()

print("✓ 헬퍼 함수 정의 완료!")

## 5. VQ-GAN 학습

In [ ]:
print("\n" + "="*80)
print("VQ-GAN 학습 시작")
print("="*80 + "\n")

# 모델 초기화
vqgan = VQGAN(
    in_channels=3,
    hidden_dims=[128, 256, 512],
    latent_dim=256,
    num_embeddings=1024
).to(CONFIG['device'])

discriminator_vq = VQGANDiscriminator(in_channels=3).to(CONFIG['device'])
discriminator_vq.apply(weights_init)

print(f"VQ-GAN 파라미터: {sum(p.numel() for p in vqgan.parameters())/1e6:.2f}M")

# 옵티마이저
opt_vq = optim.Adam(
    list(vqgan.encoder.parameters()) + 
    list(vqgan.decoder.parameters()) + 
    list(vqgan.quantizer.parameters()),
    lr=CONFIG['lr'], betas=(0.5, 0.9)
)

opt_disc_vq = optim.Adam(
    discriminator_vq.parameters(),
    lr=CONFIG['lr'], betas=(0.5, 0.9)
)

# 데이터 로더
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG['batch_size_vqgan'],
    shuffle=True,
    num_workers=CONFIG['num_workers']
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG['batch_size_vqgan'],
    shuffle=False,
    num_workers=CONFIG['num_workers']
)

perceptual_loss = SimpleLPIPS().to(CONFIG['device'])

# 학습 히스토리
vqgan_history = {
    'train_loss_gen': [],
    'train_loss_disc': [],
    'train_recon_loss': [],
    'train_vq_loss': [],
    'val_loss': [],
    'val_psnr': [],
    'val_ssim': []
}

global_step = 0

# 학습 루프
for epoch in range(CONFIG['num_epochs']):
    vqgan.train()
    discriminator_vq.train()
    
    epoch_metrics = {'loss_gen': 0, 'loss_disc': 0, 'recon_loss': 0, 'vq_loss': 0}
    
    pbar = tqdm(train_loader, desc=f"VQ-GAN Epoch {epoch+1}/{CONFIG['num_epochs']}")
    
    for batch in pbar:
        images = batch[0].to(CONFIG['device'])
        
        # Train Generator
        opt_vq.zero_grad()
        recon, vq_loss, _ = vqgan(images)
        
        recon_loss = torch.abs(images - recon).mean()
        p_loss = perceptual_loss(images, recon)
        
        if global_step > CONFIG['disc_start']:
            logits_fake = discriminator_vq(recon)
            g_loss = -torch.mean(logits_fake)
            disc_factor = CONFIG['disc_weight']
        else:
            g_loss = torch.tensor(0.0, device=CONFIG['device'])
            disc_factor = 0.0
        
        loss_gen = recon_loss + CONFIG['perceptual_weight'] * p_loss + disc_factor * g_loss + vq_loss
        loss_gen.backward()
        opt_vq.step()
        
        # Train Discriminator
        if global_step > CONFIG['disc_start']:
            opt_disc_vq.zero_grad()
            logits_real = discriminator_vq(images.detach())
            logits_fake = discriminator_vq(recon.detach())
            disc_loss = hinge_d_loss(logits_real, logits_fake)
            disc_loss.backward()
            opt_disc_vq.step()
        else:
            disc_loss = torch.tensor(0.0)
        
        epoch_metrics['loss_gen'] += loss_gen.item()
        epoch_metrics['loss_disc'] += disc_loss.item()
        epoch_metrics['recon_loss'] += recon_loss.item()
        epoch_metrics['vq_loss'] += vq_loss.item()
        
        pbar.set_postfix({
            'gen': f"{loss_gen.item():.4f}",
            'disc': f"{disc_loss.item():.4f}"
        })
        
        global_step += 1
    
    # Epoch 평균
    for key in epoch_metrics:
        epoch_metrics[key] /= len(train_loader)
    
    vqgan_history['train_loss_gen'].append(epoch_metrics['loss_gen'])
    vqgan_history['train_loss_disc'].append(epoch_metrics['loss_disc'])
    vqgan_history['train_recon_loss'].append(epoch_metrics['recon_loss'])
    vqgan_history['train_vq_loss'].append(epoch_metrics['vq_loss'])
    
    # Validation
    vqgan.eval()
    val_loss = 0
    val_psnr = 0
    val_ssim = 0
    
    with torch.no_grad():
        for i, batch in enumerate(val_loader):
            if i >= 50:  # 빠른 검증
                break
            images = batch[0].to(CONFIG['device'])
            recon, vq_loss, _ = vqgan(images)
            
            val_loss += (torch.abs(images - recon).mean() + vq_loss).item()
            val_psnr += calculate_psnr(images, recon).item()
            val_ssim += calculate_ssim(images, recon).item()
    
    val_loss /= min(50, len(val_loader))
    val_psnr /= min(50, len(val_loader))
    val_ssim /= min(50, len(val_loader))
    
    vqgan_history['val_loss'].append(val_loss)
    vqgan_history['val_psnr'].append(val_psnr)
    vqgan_history['val_ssim'].append(val_ssim)
    
    print(f"\nEpoch {epoch+1}: Val Loss={val_loss:.4f}, PSNR={val_psnr:.2f}, SSIM={val_ssim:.4f}\n")
    
    # 재구성 샘플 보기
    if (epoch + 1) % CONFIG['save_interval'] == 0:
        with torch.no_grad():
            sample_images = next(iter(val_loader))[0][:8].to(CONFIG['device'])
            sample_recon, _, _ = vqgan(sample_images)
            show_reconstruction(sample_images, sample_recon)

# 체크포인트 저장
Path('checkpoints').mkdir(exist_ok=True)
torch.save({
    'model': vqgan.state_dict(),
    'discriminator': discriminator_vq.state_dict(),
    'history': vqgan_history
}, 'checkpoints/vqgan_trained.pt')

print("\n✓ VQ-GAN 학습 완료!")

## 6. Efficient-VQGAN 학습

In [ ]:
print("\n" + "="*80)
print("Efficient-VQGAN 학습 시작")
print("="*80 + "\n")

# 모델 초기화
efficient_vqgan = EfficientVQGAN(
    in_channels=3,
    hidden_dims=[64, 128, 256],
    latent_dim=256,
    num_codebooks=4,
    codebook_size=256
).to(CONFIG['device'])

discriminator_eff = MultiScaleDiscriminator(in_channels=3, num_scales=3).to(CONFIG['device'])

print(f"Efficient-VQGAN 파라미터: {sum(p.numel() for p in efficient_vqgan.parameters())/1e6:.2f}M")

# 옵티마이저
opt_eff = optim.Adam(
    list(efficient_vqgan.encoder.parameters()) + 
    list(efficient_vqgan.decoder.parameters()) + 
    list(efficient_vqgan.quantizer.parameters()),
    lr=CONFIG['lr'], betas=(0.5, 0.9)
)

opt_disc_eff = optim.Adam(
    discriminator_eff.parameters(),
    lr=CONFIG['lr'], betas=(0.5, 0.9)
)

# 데이터 로더
train_loader_eff = DataLoader(
    train_dataset,
    batch_size=CONFIG['batch_size_efficient'],
    shuffle=True,
    num_workers=CONFIG['num_workers']
)

val_loader_eff = DataLoader(
    val_dataset,
    batch_size=CONFIG['batch_size_efficient'],
    shuffle=False,
    num_workers=CONFIG['num_workers']
)

# 학습 히스토리
eff_history = {
    'train_loss_ae': [],
    'train_loss_disc': [],
    'train_recon_loss': [],
    'train_vq_loss': [],
    'val_loss': [],
    'val_psnr': [],
    'val_ssim': []
}

global_step = 0

# 학습 루프
for epoch in range(CONFIG['num_epochs']):
    efficient_vqgan.train()
    discriminator_eff.train()
    
    epoch_metrics = {'loss_ae': 0, 'loss_disc': 0, 'recon_loss': 0, 'vq_loss': 0}
    
    pbar = tqdm(train_loader_eff, desc=f"Eff-VQGAN Epoch {epoch+1}/{CONFIG['num_epochs']}")
    
    for batch in pbar:
        images = batch[0].to(CONFIG['device'])
        
        # Train Generator
        opt_eff.zero_grad()
        recon, vq_loss, _ = efficient_vqgan(images)
        
        recon_loss = torch.abs(images - recon).mean()
        p_loss = perceptual_loss(images, recon)
        
        if global_step > CONFIG['disc_start']:
            logits_fake = discriminator_eff(recon)
            g_loss = -sum([torch.mean(logit) for logit in logits_fake]) / len(logits_fake)
            disc_factor = CONFIG['disc_weight']
        else:
            g_loss = torch.tensor(0.0, device=CONFIG['device'])
            disc_factor = 0.0
        
        loss_ae = recon_loss + CONFIG['perceptual_weight'] * p_loss + disc_factor * g_loss + vq_loss
        loss_ae.backward()
        torch.nn.utils.clip_grad_norm_(efficient_vqgan.parameters(), max_norm=1.0)
        opt_eff.step()
        
        # Train Discriminator
        if global_step > CONFIG['disc_start']:
            opt_disc_eff.zero_grad()
            logits_real = discriminator_eff(images.detach())
            logits_fake = discriminator_eff(recon.detach())
            
            disc_loss = 0
            for lr, lf in zip(logits_real, logits_fake):
                disc_loss += hinge_d_loss(lr, lf)
            disc_loss = disc_loss / len(logits_real)
            
            disc_loss.backward()
            opt_disc_eff.step()
        else:
            disc_loss = torch.tensor(0.0)
        
        epoch_metrics['loss_ae'] += loss_ae.item()
        epoch_metrics['loss_disc'] += disc_loss.item()
        epoch_metrics['recon_loss'] += recon_loss.item()
        epoch_metrics['vq_loss'] += vq_loss.item()
        
        pbar.set_postfix({
            'ae': f"{loss_ae.item():.4f}",
            'disc': f"{disc_loss.item():.4f}"
        })
        
        global_step += 1
    
    # Epoch 평균
    for key in epoch_metrics:
        epoch_metrics[key] /= len(train_loader_eff)
    
    eff_history['train_loss_ae'].append(epoch_metrics['loss_ae'])
    eff_history['train_loss_disc'].append(epoch_metrics['loss_disc'])
    eff_history['train_recon_loss'].append(epoch_metrics['recon_loss'])
    eff_history['train_vq_loss'].append(epoch_metrics['vq_loss'])
    
    # Validation
    efficient_vqgan.eval()
    val_loss = 0
    val_psnr = 0
    val_ssim = 0
    
    with torch.no_grad():
        for i, batch in enumerate(val_loader_eff):
            if i >= 50:
                break
            images = batch[0].to(CONFIG['device'])
            recon, vq_loss, _ = efficient_vqgan(images)
            
            val_loss += (torch.abs(images - recon).mean() + vq_loss).item()
            val_psnr += calculate_psnr(images, recon).item()
            val_ssim += calculate_ssim(images, recon).item()
    
    val_loss /= min(50, len(val_loader_eff))
    val_psnr /= min(50, len(val_loader_eff))
    val_ssim /= min(50, len(val_loader_eff))
    
    eff_history['val_loss'].append(val_loss)
    eff_history['val_psnr'].append(val_psnr)
    eff_history['val_ssim'].append(val_ssim)
    
    print(f"\nEpoch {epoch+1}: Val Loss={val_loss:.4f}, PSNR={val_psnr:.2f}, SSIM={val_ssim:.4f}\n")
    
    # 재구성 샘플 보기
    if (epoch + 1) % CONFIG['save_interval'] == 0:
        with torch.no_grad():
            sample_images = next(iter(val_loader_eff))[0][:8].to(CONFIG['device'])
            sample_recon, _, _ = efficient_vqgan(sample_images)
            show_reconstruction(sample_images, sample_recon)

# 체크포인트 저장
torch.save({
    'model': efficient_vqgan.state_dict(),
    'discriminator': discriminator_eff.state_dict(),
    'history': eff_history
}, 'checkpoints/efficient_vqgan_trained.pt')

print("\n✓ Efficient-VQGAN 학습 완료!")

## 7. 학습 곡선 비교

In [ ]:
# 학습 곡선 시각화
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

epochs = list(range(1, CONFIG['num_epochs'] + 1))

# Loss comparison
axes[0, 0].plot(epochs, vqgan_history['train_loss_gen'], 'o-', label='VQ-GAN', linewidth=2)
axes[0, 0].plot(epochs, eff_history['train_loss_ae'], 's-', label='Efficient-VQGAN', linewidth=2)
axes[0, 0].set_title('Generator/Autoencoder Loss', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Discriminator loss
axes[0, 1].plot(epochs, vqgan_history['train_loss_disc'], 'o-', label='VQ-GAN', linewidth=2)
axes[0, 1].plot(epochs, eff_history['train_loss_disc'], 's-', label='Efficient-VQGAN', linewidth=2)
axes[0, 1].set_title('Discriminator Loss', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Reconstruction loss
axes[0, 2].plot(epochs, vqgan_history['train_recon_loss'], 'o-', label='VQ-GAN', linewidth=2)
axes[0, 2].plot(epochs, eff_history['train_recon_loss'], 's-', label='Efficient-VQGAN', linewidth=2)
axes[0, 2].set_title('Reconstruction Loss', fontsize=14, fontweight='bold')
axes[0, 2].set_xlabel('Epoch')
axes[0, 2].set_ylabel('Loss')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

# Val loss
axes[1, 0].plot(epochs, vqgan_history['val_loss'], 'o-', label='VQ-GAN', linewidth=2)
axes[1, 0].plot(epochs, eff_history['val_loss'], 's-', label='Efficient-VQGAN', linewidth=2)
axes[1, 0].set_title('Validation Loss', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Loss')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# PSNR
axes[1, 1].plot(epochs, vqgan_history['val_psnr'], 'o-', label='VQ-GAN', linewidth=2)
axes[1, 1].plot(epochs, eff_history['val_psnr'], 's-', label='Efficient-VQGAN', linewidth=2)
axes[1, 1].set_title('Validation PSNR', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('PSNR (dB)')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# SSIM
axes[1, 2].plot(epochs, vqgan_history['val_ssim'], 'o-', label='VQ-GAN', linewidth=2)
axes[1, 2].plot(epochs, eff_history['val_ssim'], 's-', label='Efficient-VQGAN', linewidth=2)
axes[1, 2].set_title('Validation SSIM', fontsize=14, fontweight='bold')
axes[1, 2].set_xlabel('Epoch')
axes[1, 2].set_ylabel('SSIM')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ 학습 곡선 저장: training_comparison.png")

## 8. 성능 벤치마크

In [ ]:
print("\n" + "="*80)
print("성능 벤치마크 실행")
print("="*80 + "\n")

# 추론 속도 측정
def measure_inference_time(model, input_size=(1, 3, 256, 256), num_runs=100):
    model.eval()
    device = next(model.parameters()).device
    dummy = torch.randn(input_size).to(device)
    
    # Warmup
    with torch.no_grad():
        for _ in range(10):
            _ = model(dummy)
    
    # Measure
    if device.type == 'cuda':
        torch.cuda.synchronize()
    
    times = []
    with torch.no_grad():
        for _ in range(num_runs):
            start = time.time()
            _ = model(dummy)
            if device.type == 'cuda':
                torch.cuda.synchronize()
            times.append(time.time() - start)
    
    return np.mean(times) * 1000, np.std(times) * 1000

# 벤치마크 실행
results = {}

# VQ-GAN
print("VQ-GAN 벤치마크...")
inf_time, inf_std = measure_inference_time(vqgan)
results['VQ-GAN'] = {
    'params_M': sum(p.numel() for p in vqgan.parameters()) / 1e6,
    'inference_ms': inf_time,
    'inference_std': inf_std,
    'fps': 1000 / inf_time,
    'final_val_loss': vqgan_history['val_loss'][-1],
    'final_psnr': vqgan_history['val_psnr'][-1],
    'final_ssim': vqgan_history['val_ssim'][-1]
}

# Efficient-VQGAN
print("Efficient-VQGAN 벤치마크...")
inf_time, inf_std = measure_inference_time(efficient_vqgan)
results['Efficient-VQGAN'] = {
    'params_M': sum(p.numel() for p in efficient_vqgan.parameters()) / 1e6,
    'inference_ms': inf_time,
    'inference_std': inf_std,
    'fps': 1000 / inf_time,
    'final_val_loss': eff_history['val_loss'][-1],
    'final_psnr': eff_history['val_psnr'][-1],
    'final_ssim': eff_history['val_ssim'][-1]
}

# 결과 테이블
df_results = pd.DataFrame(results).T
df_results.columns = ['Parameters (M)', 'Inference (ms)', 'Std (ms)', 'FPS', 'Val Loss', 'PSNR (dB)', 'SSIM']

print("\n" + "="*80)
print("벤치마크 결과")
print("="*80)
print(df_results.to_string())
print("="*80)

# CSV 저장
df_results.to_csv('benchmark_results.csv')
print("\n✓ 결과 저장: benchmark_results.csv")

## 9. 최종 비교 시각화

In [ ]:
# 종합 비교 차트
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

models = ['VQ-GAN', 'Efficient-VQGAN']
colors = ['#3498db', '#2ecc71']

# 1. Parameters
ax1 = fig.add_subplot(gs[0, 0])
params = [results[m]['params_M'] for m in models]
bars = ax1.bar(models, params, color=colors)
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.2f}M', ha='center', va='bottom', fontweight='bold')
ax1.set_title('Model Size', fontsize=12, fontweight='bold')
ax1.set_ylabel('Parameters (M)')
ax1.grid(axis='y', alpha=0.3)

# 2. Inference Time
ax2 = fig.add_subplot(gs[0, 1])
inf_time = [results[m]['inference_ms'] for m in models]
bars = ax2.bar(models, inf_time, color=colors)
for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.2f}ms', ha='center', va='bottom', fontweight='bold')
ax2.set_title('Inference Time', fontsize=12, fontweight='bold')
ax2.set_ylabel('Time (ms)')
ax2.grid(axis='y', alpha=0.3)

# 3. FPS
ax3 = fig.add_subplot(gs[0, 2])
fps = [results[m]['fps'] for m in models]
bars = ax3.bar(models, fps, color=colors)
for bar in bars:
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f}', ha='center', va='bottom', fontweight='bold')
ax3.set_title('Throughput', fontsize=12, fontweight='bold')
ax3.set_ylabel('FPS')
ax3.grid(axis='y', alpha=0.3)

# 4. PSNR
ax4 = fig.add_subplot(gs[1, 0])
psnr = [results[m]['final_psnr'] for m in models]
bars = ax4.bar(models, psnr, color=colors)
for bar in bars:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.2f}', ha='center', va='bottom', fontweight='bold')
ax4.set_title('PSNR (Higher is Better)', fontsize=12, fontweight='bold')
ax4.set_ylabel('PSNR (dB)')
ax4.grid(axis='y', alpha=0.3)

# 5. SSIM
ax5 = fig.add_subplot(gs[1, 1])
ssim = [results[m]['final_ssim'] for m in models]
bars = ax5.bar(models, ssim, color=colors)
for bar in bars:
    height = bar.get_height()
    ax5.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.4f}', ha='center', va='bottom', fontweight='bold')
ax5.set_title('SSIM (Higher is Better)', fontsize=12, fontweight='bold')
ax5.set_ylabel('SSIM')
ax5.grid(axis='y', alpha=0.3)

# 6. Val Loss
ax6 = fig.add_subplot(gs[1, 2])
val_loss = [results[m]['final_val_loss'] for m in models]
bars = ax6.bar(models, val_loss, color=colors)
for bar in bars:
    height = bar.get_height()
    ax6.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.4f}', ha='center', va='bottom', fontweight='bold')
ax6.set_title('Validation Loss (Lower is Better)', fontsize=12, fontweight='bold')
ax6.set_ylabel('Loss')
ax6.grid(axis='y', alpha=0.3)

# 7. Radar Chart
ax7 = fig.add_subplot(gs[2, :], projection='polar')

# Normalize metrics
categories = ['Speed\n(FPS)', 'Quality\n(PSNR)', 'Quality\n(SSIM)', 'Efficiency\n(Params)', 'Loss']

def normalize(values, inverse=False):
    vmin, vmax = min(values), max(values)
    if vmax == vmin:
        return [0.5] * len(values)
    if inverse:
        return [(vmax - v) / (vmax - vmin) for v in values]
    return [(v - vmin) / (vmax - vmin) for v in values]

vqgan_values = [
    normalize([results['VQ-GAN']['fps'], results['Efficient-VQGAN']['fps']])[0],
    normalize([results['VQ-GAN']['final_psnr'], results['Efficient-VQGAN']['final_psnr']])[0],
    normalize([results['VQ-GAN']['final_ssim'], results['Efficient-VQGAN']['final_ssim']])[0],
    normalize([results['VQ-GAN']['params_M'], results['Efficient-VQGAN']['params_M']], inverse=True)[0],
    normalize([results['VQ-GAN']['final_val_loss'], results['Efficient-VQGAN']['final_val_loss']], inverse=True)[0]
]

eff_values = [
    normalize([results['VQ-GAN']['fps'], results['Efficient-VQGAN']['fps']])[1],
    normalize([results['VQ-GAN']['final_psnr'], results['Efficient-VQGAN']['final_psnr']])[1],
    normalize([results['VQ-GAN']['final_ssim'], results['Efficient-VQGAN']['final_ssim']])[1],
    normalize([results['VQ-GAN']['params_M'], results['Efficient-VQGAN']['params_M']], inverse=True)[1],
    normalize([results['VQ-GAN']['final_val_loss'], results['Efficient-VQGAN']['final_val_loss']], inverse=True)[1]
]

angles = [n / float(len(categories)) * 2 * np.pi for n in range(len(categories))]
vqgan_values += vqgan_values[:1]
eff_values += eff_values[:1]
angles += angles[:1]

ax7.plot(angles, vqgan_values, 'o-', linewidth=2, label='VQ-GAN', color=colors[0])
ax7.fill(angles, vqgan_values, alpha=0.15, color=colors[0])
ax7.plot(angles, eff_values, 's-', linewidth=2, label='Efficient-VQGAN', color=colors[1])
ax7.fill(angles, eff_values, alpha=0.15, color=colors[1])

ax7.set_xticks(angles[:-1])
ax7.set_xticklabels(categories, size=11)
ax7.set_ylim(0, 1)
ax7.set_title('Overall Performance Comparison (Normalized)', fontsize=14, fontweight='bold', pad=20)
ax7.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=11)
ax7.grid(True)

plt.savefig('final_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ 최종 비교 차트 저장: final_comparison.png")

## 10. 최종 리포트 생성

In [ ]:
# HTML 리포트 생성
report_html = f"""
<html>
<head>
    <title>VQ-GAN vs Efficient-VQGAN 비교 리포트</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 20px; background-color: #f5f5f5; }}
        h1 {{ color: #2c3e50; border-bottom: 3px solid #3498db; padding-bottom: 10px; }}
        h2 {{ color: #34495e; margin-top: 30px; }}
        table {{ border-collapse: collapse; width: 100%; margin: 20px 0; background: white; box-shadow: 0 2px 3px rgba(0,0,0,0.1); }}
        th, td {{ border: 1px solid #ddd; padding: 12px; text-align: left; }}
        th {{ background-color: #3498db; color: white; }}
        tr:nth-child(even) {{ background-color: #f8f9fa; }}
        .winner {{ background-color: #d4edda; font-weight: bold; }}
        .metric {{ font-size: 24px; font-weight: bold; color: #2c3e50; }}
        .summary {{ background: white; padding: 20px; margin: 20px 0; border-left: 4px solid #3498db; }}
    </style>
</head>
<body>
    <h1>🔥 VQ-GAN vs Efficient-VQGAN 비교 리포트</h1>
    
    <div class="summary">
        <h2>📊 Executive Summary</h2>
        <p><strong>학습 설정:</strong> {CONFIG['num_epochs']} epochs, Dataset: {CONFIG['dataset']}</p>
        <p><strong>파라미터 감소:</strong> {(1 - results['Efficient-VQGAN']['params_M']/results['VQ-GAN']['params_M'])*100:.1f}%</p>
        <p><strong>속도 향상:</strong> {(results['Efficient-VQGAN']['fps']/results['VQ-GAN']['fps']-1)*100:.1f}%</p>
    </div>
    
    <h2>📈 성능 비교 테이블</h2>
    {df_results.to_html(classes='table')}
    
    <h2>🏆 주요 발견사항</h2>
    <div class="summary">
        <ul>
            <li><strong>모델 크기:</strong> Efficient-VQGAN이 {(1 - results['Efficient-VQGAN']['params_M']/results['VQ-GAN']['params_M'])*100:.1f}% 더 작음</li>
            <li><strong>추론 속도:</strong> Efficient-VQGAN이 {'더 빠름' if results['Efficient-VQGAN']['fps'] > results['VQ-GAN']['fps'] else '더 느림'} ({abs((results['Efficient-VQGAN']['fps']/results['VQ-GAN']['fps']-1)*100):.1f}%)</li>
            <li><strong>이미지 품질 (PSNR):</strong> {'VQ-GAN' if results['VQ-GAN']['final_psnr'] > results['Efficient-VQGAN']['final_psnr'] else 'Efficient-VQGAN'}이 {abs(results['VQ-GAN']['final_psnr'] - results['Efficient-VQGAN']['final_psnr']):.2f} dB 우세</li>
            <li><strong>전체 평가:</strong> Efficient-VQGAN은 효율성과 품질의 좋은 균형을 제공</li>
        </ul>
    </div>
    
    <h2>📅 생성 정보</h2>
    <p>생성 일시: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
</body>
</html>
"""

with open('comparison_report.html', 'w', encoding='utf-8') as f:
    f.write(report_html)

print("✓ HTML 리포트 저장: comparison_report.html")

# 노트북에서 리포트 표시
display(HTML(report_html))

## 11. 완료 요약

In [ ]:
print("\n" + "="*80)
print("🎉 전체 파이프라인 완료!")
print("="*80)
print("\n생성된 파일:")
print("  ✓ checkpoints/vqgan_trained.pt          - VQ-GAN 체크포인트")
print("  ✓ checkpoints/efficient_vqgan_trained.pt - Efficient-VQGAN 체크포인트")
print("  ✓ training_comparison.png                - 학습 곡선 비교")
print("  ✓ final_comparison.png                   - 최종 성능 비교")
print("  ✓ benchmark_results.csv                  - 벤치마크 결과")
print("  ✓ comparison_report.html                 - HTML 리포트")
print("\n" + "="*80)
print("\n주요 결과:")
print(f"  • VQ-GAN 파라미터:          {results['VQ-GAN']['params_M']:.2f}M")
print(f"  • Efficient-VQGAN 파라미터: {results['Efficient-VQGAN']['params_M']:.2f}M")
print(f"  • 파라미터 감소:            {(1 - results['Efficient-VQGAN']['params_M']/results['VQ-GAN']['params_M'])*100:.1f}%")
print(f"  • VQ-GAN FPS:               {results['VQ-GAN']['fps']:.2f}")
print(f"  • Efficient-VQGAN FPS:      {results['Efficient-VQGAN']['fps']:.2f}")
print(f"  • 속도 차이:                {abs((results['Efficient-VQGAN']['fps']/results['VQ-GAN']['fps']-1)*100):.1f}%")
print("\n" + "="*80 + "\n")